# bounding-boxes

Aubrey Moore 2020-07-14

Outputs images displaying contents of bounding boxes for each track.

Run papermill and nbconvert using something like the following. Results will be in the HTML file.
```
mkdir reports

mkdir reports/06-23-test-cut-detector-20

papermill --prepare-only bounding-boxes.ipynb reports/06-23-test-cut-detector-20/06-23-test-cut-detector-20.ipynb -y "{'DATADIR':'/media/aubrey/9C33-6BBD/06-23-test-cut-detector-20','CVATXMLFILE':'06-23-test-cut-detector-20.xml','VIDEOFILE':'20200623_processed.mp4','SAVE_BB_IMAGES':'False'}"

jupyter nbconvert --execute --to html reports/06-23-test-cut-detector-20/06-23-test-cut-detector-20.ipynb

```

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import cv2
import sys
import glob
import numpy as np
from IPython.display import HTML, Image

In [ ]:
DATADIR = '/media/aubrey/9C33-6BBD/06-23-test-cut-detector-20'
CVATXMLFILE = '06-23-test-cut-detector-20.xml'
VIDEOFILE = '20200623_processed.mp4'
SAVE_BB_IMAGES = 'False'

In [ ]:
# Parameters
CVATXMLFILE = "06-23-test-cut-detector-20.xml"
DATADIR = "/media/aubrey/9C33-6BBD/06-23-test-cut-detector-20"
SAVE_BB_IMAGES = "False"
VIDEOFILE = "20200623_processed.mp4"


In [ ]:
def str2int(s):
    return int(float(s))

# str2int('7.53')

In [ ]:
def get_bounding_boxes(xml_file):
    """
    Returns a pandas dataframe.
    """   
    tree = ET.parse(xml_file)
    root = tree.getroot()

    mylist = []
    for track in root.findall('track'):
        for box in track.findall('box'):
            mydict = box.attrib
            mydict.update(track.attrib)
            mylist.append(mydict)
    df = pd.DataFrame(mylist)
    df = df[(df.occluded=='0') & (df.outside=='0')]
    df.drop(['occluded', 'outside'], axis=1, inplace=True)
    df.rename({'id': 'track_id'}, axis=1, inplace=True)
    df.xbr = df.xbr.apply(lambda x: str2int(x))
    df.xtl = df.xtl.apply(lambda x: str2int(x))
    df.ybr = df.ybr.apply(lambda x: str2int(x))
    df.ytl = df.ytl.apply(lambda x: str2int(x))
    df.frame = df.frame.apply(lambda x: str2int(x))
    df.track_id = df.track_id.apply(lambda x: str2int(x))
    return df

#get_bounding_boxes(xml_file)

In [ ]:
def write_bounding_box_image(row):
    """
    Writes image contained in bounding box.
    """
    cap = cv2.VideoCapture(f'{DATADIR}/{VIDEOFILE}')
    cap.set(1, row.frame)
    _, frame = cap.read()
    img = frame[row.ytl:row.ybr, row.xtl:row.xbr]
    blank[row.ytl:row.ybr, row.xtl:row.xbr] = img
    cap.release()
    if SAVE_BB_IMAGES=='True':
        cv2.imwrite(f't{row.track_id:04d}f{row.frame:04d}.jpg', img)   

In [ ]:
def create_blank(width, height, rgb_color=(0,0,0)):
    """Create new image(numpy array) filled with certain color in RGB"""
    # Create black blank image
    image = np.zeros((height, width, 3), np.uint8)

    # Since OpenCV uses BGR, convert the color first
    color = tuple(reversed(rgb_color))
    
    # Fill image with color
    image[:] = color

    return image

In [ ]:
def _src_from_data(data):
    """Base64 encodes image bytes for inclusion in an HTML img element"""
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

def gallery(images, row_height='auto'):
    """Shows a set of images in a gallery that flexes with the width of the notebook.
    
    Parameters
    ----------
    images: list of str or bytes
        URLs or bytes of images to display

    row_height: str
        CSS height value to assign to all images. Set to 'auto' by default to show images
        with their native dimensions. Set to a value like '250px' to make all rows
        in the gallery equal height.
    """
    figures = []
    for image in images:
        if isinstance(image, bytes):
            src = _src_from_data(image)
            caption = ''
        else:
            src = image
            caption = f'<figcaption style="font-size: 0.6em">{image}</figcaption>'
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{src}" style="height: {row_height}">
              {caption}
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

# MAIN

In [ ]:
dfbb = get_bounding_boxes(f'{DATADIR}/{CVATXMLFILE}')
grouped = dfbb.groupby('track_id')
blankfilelist = []
for track_id, df in grouped:
    blank = create_blank(1920, 1080, (128,128,128))       
    df.apply(write_bounding_box_image, axis=1)
    blankfile = f't{track_id:04d}.jpg'
    blankfilelist.append(blankfile)
    cv2.imwrite(blankfile, blank)
gallery(blankfilelist)